In [ ]:
import re
import subprocess
import json
import glob
import argparse
import tempfile
import os

In [ ]:
def replace_in_ipynb(file_path, temp, patterns, replacements):
    # Open the file
    with open(file_path, "r") as file:
        # Read the contents of the file and parse it as JSON
        notebook = json.load(file)

    # Replace the desired parts of the notebook
    for i in range(len(patterns)):
        pattern = patterns[i]
        replacement = replacements[i]
        for cell in notebook["cells"]:
            if "source" in cell:
                cell["source"] = [re.sub(pattern, replacement, line) for line in cell["source"]]

    # Write the modified notebook to the temporary file
    with open(temp, "w") as file:
        json.dump(notebook, file)

In [ ]:
template_path = "/home/berki/Software/TPCQCVis/TPCQCVis/reports/TPC_asyncQC_template.ipynb"
# Create a temporary file with a unique filename for the run report
with tempfile.NamedTemporaryFile(prefix="TPCQC_", suffix=".ipynb", delete=False) as temp_run:
    temp_run_path = temp_run.name

In [ ]:
path = "/cave/alice-tpc-qc/data/2023/"
periods = ["LHC23"+name for name in [#"h","j","k","m","p","q",
    "v","y","z","zc","zl","zs"]]
apass = "apass1"
runNumber = 132132

for period in periods:
    print(f"Creating report for period {period}")
    replace_in_ipynb(template_path, temp_run_path,
            ["myPeriod", "myPass", "123456", "myPath"],
            [period, apass, str(runNumber), path]
        )
    
    # The command and its arguments for the run report
    run_report_command = [
        "jupyter", "nbconvert", temp_run_path, "--to", "html", "--template", "classic", "--no-input", "--execute",
        "--output", "/cave/alice-tpc-qc/reports/2023/" + period + "/"+ period+"_apass1_comparison.html"
    ]
    
    # Run the command for the run report
    output = subprocess.run(run_report_command, capture_output=True)